In [74]:
import easyocr
from PIL import Image
import openai
import os
from dotenv import load_dotenv
import requests

In [93]:
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [94]:
image_path = "../Back-end/sample/Test1.png"  

reader = easyocr.Reader(['th', 'en'])
results = reader.readtext(image_path, detail=1)  

for bbox, text, _ in results:
    print(f"{text} -> {bbox}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


ลำดับ -> [[51, 23], [89, 23], [89, 43], [51, 43]]
รายการ -> [[179, 27], [227, 27], [227, 41], [179, 41]]
หน่วยนับ -> [[320, 16], [378, 16], [378, 40], [320, 40]]
ตองการเมือ -> [[395, 17], [465, 17], [465, 37], [395, 37]]
ปรมาณ -> [[483, 19], [529, 19], [529, 35], [483, 35]]
ภาคาตอ -> [[577, 19], [625, 19], [625, 35], [577, 35]]
เป็นเง่น -> [[701, 15], [747, 15], [747, 33], [701, 33]]
รวมทั่งสืน -> [[924, 6], [984, 6], [984, 30], [924, 30]]
ทนวย -> [[581, 45], [619, 45], [619, 59], [581, 59]]
มูลค่าเพิ่ม -> [[802, 30], [867, 30], [867, 60], [802, 60]]
ชัวโมง -> [[329, 67], [369, 67], [369, 87], [329, 87]]
28 ก.พ.68 -> [[395, 69], [461, 69], [461, 87], [395, 87]]
1oo.00 -> [[607, 67], [653, 67], [653, 83], [607, 83]]
1,500.00 -> [[719, 65], [777, 65], [777, 83], [719, 83]]
d.qd -> [[851, 63], [881, 63], [881, 79], [851, 79]]
1,500.00 -> [[947, 61], [1005, 61], [1005, 79], [947, 79]]
 จางเหมาบริการรถตูพรอมคนข์บรถ -> [[104, 72], [302, 72], [302, 96], [104, 96]]
บอกเวลา หมายเลขทะเบียน -> [[

In [95]:
target_keywords = ['รวม', 'ยอดสุทธิ', 'รวมทั้งสิ้น', 'ภาษี', 'จำนวนเงินสุทธิ']
target_boxes = []

for bbox, text, _ in results:
    if any(keyword in text for keyword in target_keywords):
        target_boxes.append(bbox)


In [96]:
# ขยายพื้นที่รอบกล่องข้อความ "ยอดรวม"
padding_x = 150            # ซ้าย/ขวา
padding_y_top = 40         # ขอบบน
padding_y_bottom = 150    # ✅ เพิ่มเยอะขึ้น เพื่อให้จับยอดรวมเต็ม

img = Image.open(image_path)

if target_boxes:
    x_min = min([min(p[0] for p in box) for box in target_boxes])
    y_min = min([min(p[1] for p in box) for box in target_boxes])
    x_max = max([max(p[0] for p in box) for box in target_boxes])
    y_max = max([max(p[1] for p in box) for box in target_boxes])

    cropped_img = img.crop((
        max(x_min - padding_x, 0),
        max(y_min - padding_y_top, 0),
        min(x_max + padding_x, img.width),
        min(y_max + padding_y_bottom, img.height)
    ))

    cropped_img.show()
else:
    print("❌ ไม่พบตำแหน่งที่เกี่ยวกับยอดรวม")


In [97]:
user_prompt = """
จากข้อความ OCR ด้านล่างนี้ กรุณาดึงเฉพาะยอดรวมสุทธิที่พิมพ์ไว้ในเอกสาร โดยเน้นเฉพาะยอดที่เอกสารได้คำนวณไว้แล้วและแสดงไว้อย่างชัดเจน

✅ ยอดที่ต้องการ:
- ยอดรวมสุทธิที่รวมภาษีมูลค่าเพิ่มแล้ว (หากมี)
- มักอยู่บริเวณบรรทัดสุดท้ายของตารางค่าใช้จ่าย หรือหัวข้อ “จำนวนเงินสุทธิตัวอักษร” หรือคล้ายกัน

❌ ห้ามเลือก:
- ตัวเลขในส่วนหมายเหตุ หรือคำอธิบายท้ายเอกสาร
- ตัวเลขที่คุณต้องคำนวณเองจากรายการต่าง ๆ
- ยอดรวมที่ไม่ใช่ของเอกสารนี้โดยตรง เช่น ยอดรวมของหลายใบเสร็จรวมกัน

หากไม่พบยอดรวมชัดเจน โปรดระบุว่า “ไม่พบยอดรวมที่ชัดเจนในเอกสารนี้”
"""

prompt = f"""
คุณคือผู้ช่วยอ่านข้อความจากใบเสร็จ

ข้อความที่ได้จาก OCR คือ:

{text}

{user_prompt}
"""


In [98]:
import requests

headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "HTTP-Referer": "https://localhost",  # หรือชื่อโปรเจกต์ถ้ามี
    "X-Title": "ReadReciptGPT"
}

data = {
    "model": "openai/gpt-3.5-turbo",
    "messages": [
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers=headers,
    json=data
)

response_json = response.json()
answer = response_json["choices"][0]["message"]["content"]
print("✅ คำตอบที่ได้จาก GPT:", answer)


✅ คำตอบที่ได้จาก GPT: ไม่พบยอดรวมที่ชัดเจนในเอกสารนี้  ในที่นี้ไม่สามารถดึงยอดรวมสุทธิที่รวมภาษีมูลค่าเพิ่มได้ ต้องขออภัยด้วยว่าไม่สามารถช่วยเหลือได้ในเรื่องนี้ครับ/ค่ะ หากต้องการความช่วยเหลือในเรื่องอื่น ๆ กรุณาบอกมานะครับ/ค่ะ 🙏😊 ขอบคุณครับ/ค่ะ
